In [2]:
# !pip install excel2json-3
# !pip install python_jwt
# !pip install gcloud
# !pip install sseclient
# !pip install Crypto
# !pip install requests_toolbelt
# !pip install firebase
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !pip install requests
# !pip install html5lib
# !pip install bs4
# !pip install selenium
# !pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
import schedule
import time
from selenium import webdriver              
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import io
import re
import os
from firebase import firebase
import xlwt 
from xlwt import Workbook 
import xlrd
from selenium import webdriver
from firebase import firebase
import excel2json
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
sys.path.insert(0,'./chromedriver')

def details(url, list_=False,city=None,type_=None):
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get(url)
    page_soup = BeautifulSoup(driver.page_source,'lxml');
    
    if list_ == True:
        abc = get_abc(page_soup,city,type_)
    else:
        abc = get_events_url(page_soup)
    driver.close()
    return abc

def get_events_url(soup):
    list_url =[]
    li_con=soup.find_all('li',{'class':'item'})
    for li in li_con:
    # print(li_con[0])
        for a in li.find_all('a', href=True):
          # print)
          list_url.append((a['href']))
    return list_url
  # # Then search in that div_container for all p tags with class "hello"
  # for ptag in div_container.find_all('p', class_='hello'):
  # # prints the p tag content
  # print(ptag.text
  # print(event_name[0])
def get_abc(soup,city,type_):
    # use ur tags here
#     event_name=soup.find('div',{'class':'pd-lr-10 span9 fullwidth'})
    event_name=soup.find('h1',{'class':'overlay-h1'})
    

    if event_name == None:
        event_name='none'
    else:
        event_name=event_name.text

    event_time=soup.find('span',{'class':'display_start_time'})
    
    if event_time == None:
        event_time='none'
    else:
        event_time = event_time.get_text()
    event_time = event_time.strip()

    event_city=city
    event_type=type_
    event_picture=soup.find('img', {'src':re.compile('.jpg')}) 

    if event_picture == None:
        event_picture='none'
    else:
        event_picture=event_picture['src']+'\n' 
  
    venue=soup.find('span',{'class':'full-venue'})

    if venue == None:
        venue='none'
    else:
        venue=venue.text

    event_description=soup.find('div',{'class':'event-description-html'})

    if event_description == None:
        event_description='none'
    else:
        event_description=event_description.text
  
    # source=soup.find('li',{'class':'orgInfoBlock'})

    source=soup.find('a',{'class':'toh orglink'})
    if (source != None):
        source=source.text
    lines=[event_name,event_time,venue,event_description,source,event_picture,event_type,event_city]
    return lines

def start_scarping(): 
    # Workbook is created 
    wb2 = Workbook() 
    # add_sheet is used to create sheet. 
    sheet1 = wb2.add_sheet('Event list') 
    sheet1.write(0, 0, 'Event name')
    sheet1.write(0, 1, 'Event time') 
    sheet1.write(0, 2, 'Venue') 
    sheet1.write(0, 3, 'Description') 
    sheet1.write(0, 4, 'Source')
    sheet1.write(0, 5, 'Picture')
    sheet1.write(0, 6, 'Type')
    sheet1.write(0, 7, 'City')
    wb2.save('Events_test.xls')
    i=1
    place=0
    places= ['islamabad','rawalpindi','karachi','multan','lahore','murree','peshawar','faisalabad','gilgit','skardu'] 
    categotries=['entertainment','food-drinks','trips-adventures','sports','business','workshops']
    all_events=[]
    while(place <len(places)):
        cat=0
        while(cat<len(categotries)):
            list_url =details('https://allevents.in/'+places[place]+'/'+categotries[cat])
            for url in list_url:
                thislist =details(url,True,places[place],categotries[cat])
                all_events.append(thislist)
                sheet1.write(i, 0, thislist[0]) 
                sheet1.write(i, 1, thislist[1])
                sheet1.write(i, 2, thislist[2]) 
                sheet1.write(i, 3, thislist[3]) 
                sheet1.write(i, 4, thislist[4]) 
                sheet1.write(i, 5, thislist[5]) 
                sheet1.write(i, 6, thislist[6]) 
                sheet1.write(i, 7, thislist[7])
                wb2.save('Events_test.xls')
                i=i+1
                print("count => ",i," category: ",cat," city:",place) 
            cat=cat+1
        place=place+1

def delete_post():
#     start_scarping()
    fb = firebase.FirebaseApplication('https://fyptrial-d98b1.firebaseio.com', None)
    db = fb.delete('/All-Events', None)
    
    workbook = xlrd.open_workbook('Events_test.xls')
    worksheet = workbook.sheet_by_name('Event list')
    column_names = worksheet.row(0)
    for rx in range(worksheet.nrows-1):
        db = fb.post('/All-Events', {
                                    'id':worksheet.cell_value(rx+1,0),
                                    'title':worksheet.cell_value(rx+1,0),
                                    'time':worksheet.cell_value(rx+1,1),
                                    'venue':worksheet.cell_value(rx+1,2),
                                    'description':worksheet.cell_value(rx+1,3),
                                    'source':worksheet.cell_value(rx+1,4),
                                    'picture':worksheet.cell_value(rx+1,5),
                                    'type':worksheet.cell_value(rx+1,6),
                                    'city':worksheet.cell_value(rx+1,7),
                                    'rated':'FALSE',
                                    'rating':-1
                                    })
def job():
    delete_post()
    
schedule.every(10).seconds.do(delete_post())
# schedule.every().day.at("01:00").do(job())
while True:
    schedule.run_pending()
    time.sleep(1) # wait one minute

'cp' is not recognized as an internal or external command,
operable program or batch file.


TypeError: the first argument must be callable

In [ ]:
#!flask/bin/python
from flask import request
from flask import Flask

app = Flask(__name__)

@app.route('/get_recommendations', methods=['POST'])
def create_task():
    if not request.json or not 'id' in request.json:
        abort(400)
    recommended_events = system(input_user_id,df_new_events, df_old_events,df_rating)
    return jsonify({'event':recommended_events})

  
@app.route('/get_recommendations')
def index():
    return "Recommended Events Get Call!"

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8000)